# Keras

In [52]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [31]:
import warnings

# suppress warnings
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=FutureWarning)
    import tensorflow as tf
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Dense, LSTM, Input, concatenate

### read data

In [3]:
raw = pd.read_csv('corn.csv')
raw = raw.append(pd.read_csv('corn_barchart_new.csv'))
raw.head()
#raw.describe()

,year,month,open,close,open_net,close_net,open_rel,close_rel,volume,openint,day,date
0,1971,12,119.500,118.625,0.875,0.00,1.007376,1.000000,4920,2915,0,1971-12-20
1,1971,12,119.500,118.625,0.875,0.00,1.007376,1.000000,4920,2915,1,1971-12-20
2,1971,12,119.500,118.625,0.875,0.00,1.007376,1.000000,4920,2915,2,1971-12-20
3,1971,12,117.875,118.875,-0.750,0.25,0.993678,1.002107,1925,3355,3,1971-12-17
4,1971,12,118.750,117.875,0.125,-0.75,1.001054,0.993678,3455,5285,4,1971-12-16


### create scalers for the data

In [4]:
# scaled data
columns = ['close']

months = pd.unique(raw['month'])

scaledtimeseries = {}
for month in months:
    scaledtimeseries[month] = {}
    years = pd.unique(raw[raw['month']==month]['year'])
    
    for year in years:
        ts = raw[(raw['year']==year) & (raw['month']==month)]
        if len(ts) > 0:
            ts = ts.copy()
            ts.sort_values(by=['date'], inplace=True)
            ts = ts.reset_index(drop=True)
            scaledtimeseries[month][year] = ts

    # transform the data for the month
    data = pd.DataFrame()
    for year_data in scaledtimeseries[month].values():
        data = data.append(year_data[columns])
    data = np.array(data)
    scaler = StandardScaler()
    scaler.fit(data)
        
    scaledtimeseries[month]['scaler'] = scaler

construct the training set.

For each year:
```
y_tgt = F(y_i, ..., y_i-w)
```

In [39]:
def mkTrainingData(df, month, backoff, window, z_month):
    X = pd.DataFrame()
    Z = pd.DataFrame()
    
    y = []
    end_day = backoff + window

    M = df[month]
    Mz = df[z_month]
    years = M.keys()
    
    scaler = M['scaler']
    zsacler = Mz['scaler']
    
    for year in years:
        if isinstance(year, str):
            continue   # only process years; skip scaler

        ts = M[year]
        # grab close prices, then reverse order
        close_prices = ts[(ts['day']>=backoff) & (ts['day']<end_day)][['close']].iloc[::-1]
        if len(close_prices) == window:
            X = X.append(close_prices)
            y.append(list(ts[['close']].iloc[-1]))
            tz = Mz[year]
            close_prices = tz[(tz['day']>=backoff) & (tz['day']<end_day)][['close']].iloc[::-1]
            Z = Z.append(close_prices)

    # scale and shape X values
    X = scaler.transform(X)
    X = np.array(X)
    X = X.flatten().reshape(X.shape[0]//(window), window, 1)

    # scale and shape Z values
    Z = scaler.transform(Z)
    Z = np.array(Z)
    Z = Z.flatten().reshape(Z.shape[0]//(window), window, 1)
    
    # scale and share y values
    y = scaler.transform(y)
    y = np.array(y)
    y = y.reshape(y.shape[0], 1)
    
    return X, y, Z

In [115]:
def split(X, y, Z, factor, seed=None):
    l = len(X)
    assert l == len(y) and l == len(Z), 'inputs must have same length'
    
    if seed:
        np.random.seed(seed)
    size = int(round(l * factor))

    # select test rows
    test_indexes = np.random.random_integers(0, l-1, size=size)
    # probably not necessary to sort, but ...
    test_indexes.sort()    
    
    all_indexes = list(range(l))
    train_indexes = [all_indexes[i] for i in range(l) if i not in test_indexes]
    
    X_train = X[train_indexes, :]
    y_train = y[train_indexes, :]
    Z_train = Z[train_indexes, :]
    
    X_test = X[test_indexes, :]
    y_test = y[test_indexes, :]
    Z_test = Z[test_indexes, :]

    return X_train, X_test, y_train, y_test, Z_train, Z_test
    # return X_train, np.ndarray(X_test), y_train, np.ndarray(y_test), Z_train, np.ndarray(Z_test)

## LSTM
Following https://medium.com/datadriveninvestor/multivariate-time-series-using-rnn-with-keras-7f78f4488679

In [112]:
month = 9
z_month = 5
window = 30
Columns = ['close']

lcols = len(Columns)

left_input = Input(shape=(window, lcols,))
left_stack = LSTM(units=window, return_sequences=True, input_shape=(window, len(columns)))
left = left_stack(left_input)
left =  LSTM(units=window, return_sequences=True)(left)
left = LSTM(units=window)(left)
left = Dense(units=15)(left)

right_input = Input(shape=(window, lcols))
right_stack = LSTM(units=window, return_sequences=True, input_shape=(window, len(columns)))
right = right_stack(right_input)
right =  LSTM(units=window, return_sequences=True)(right)
right = LSTM(units=window)(right)
right = Dense(units=15)(right)

merged = concatenate([left, right])

outputs = Dense(units=15)(merged)
'''
outputs = Dense(units=15, activation='relu')(left)
'''
outputs = Dense(units=lcols, activation='sigmoid')(outputs)

model = tf.keras.Model(inputs=(left_input, right_input), outputs=outputs)
# model.summary()
model.compile(optimizer='adam', loss='mean_squared_error')

X, y, Z = mkTrainingData(scaledtimeseries, month, backoff, window, z_month)

In [109]:
model.summary()

Model: "model_19"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_49 (InputLayer)           [(None, 30, 1)]      0                                            
__________________________________________________________________________________________________
input_50 (InputLayer)           [(None, 30, 1)]      0                                            
__________________________________________________________________________________________________
lstm_136 (LSTM)                 (None, 30, 30)       3840        input_49[0][0]                   
__________________________________________________________________________________________________
lstm_139 (LSTM)                 (None, 30, 30)       3840        input_50[0][0]                   
___________________________________________________________________________________________

In [116]:
X_train, X_test, y_train, y_test, Z_train, Z_test = split(X, y, Z, .1)
model.fit((X_train, Z_train), y_train, epochs=100, batch_size=20)

Epoch 1/100
20/54 [==========>...................] - ETA: 0s - loss: 0.7614

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: This function is deprecated. Please call randint(0, 59 + 1) instead
  # Remove the CWD from sys.path while we load stuff.


54/54 [==============================] - 0s 6ms/sample - loss: 0.7801
Epoch 2/100
54/54 [==============================] - 0s 4ms/sample - loss: 0.7793
Epoch 3/100
54/54 [==============================] - 0s 4ms/sample - loss: 0.7808
Epoch 4/100
54/54 [==============================] - 0s 4ms/sample - loss: 0.7771
Epoch 5/100
54/54 [==============================] - 0s 5ms/sample - loss: 0.7777
Epoch 6/100
54/54 [==============================] - 0s 4ms/sample - loss: 0.7781
Epoch 7/100
54/54 [==============================] - 0s 4ms/sample - loss: 0.7760
Epoch 8/100
54/54 [==============================] - 0s 5ms/sample - loss: 0.7759
Epoch 9/100
54/54 [==============================] - 0s 4ms/sample - loss: 0.7758
Epoch 10/100
54/54 [==============================] - 0s 5ms/sample - loss: 0.7752
Epoch 11/100
54/54 [==============================] - 0s 5ms/sample - loss: 0.7748
Epoch 12/100
54/54 [==============================] - 0s 8ms/sample - loss: 0.7743
Epoch 13/100
54/54 [=====

Epoch 100/100
54/54 [==============================] - 0s 4ms/sample - loss: 0.7676


In [117]:
inverse = scaledtimeseries[month]['scaler'].inverse_transform
inv = lambda x: inverse(x)[0]


y_predict = model.predict((X_test, Z_test))
print('{:>6s} {:>6s} {:>6s} {:>7s} {:>7s}'.format('X_0', 'true', 'pred', "acc", "gain"))

for i in range(len(X_test)):
    X_0, test, pred = inv(X_test[i][0]), inv(y_test[i]), inv(y_predict[i])
    print('{:6.2f} {:6.2f} {:6.2f} {:+6.2f}% {:7.2f}'.format(
        X_0, test, pred, (pred-test)/test*100, test-X_0))

   X_0   true   pred     acc    gain
219.50 266.50 292.29  +9.68%   47.00
269.25 234.25 292.29 +24.78%  -35.00
249.75 198.75 292.29 +47.06%  -51.00
137.30 113.60 292.29 +157.30%  -23.70
215.50 195.00 292.29 +49.89%  -20.50
426.75 305.00 415.02 +36.07% -121.75


In [111]:
inverse = scaledtimeseries[month]['scaler'].inverse_transform
inv = lambda x: inverse(x)[0]


y_predict = model.predict((X_test, Z_test))
print('{:>6s} {:>6s} {:>6s} {:>7s} {:>7s}'.format('X_0', 'true', 'pred', "acc", "gain"))

for i in range(len(X_test)):
    X_0, test, pred = inv(X_test[i][0]), inv(y_test[i]), inv(y_predict[i])
    print('{:6.2f} {:6.2f} {:6.2f} {:+6.2f}% {:7.2f}'.format(
        X_0, test, pred, (pred-test)/test*100, test-X_0))

   X_0   true   pred     acc    gain
190.25 179.25 292.29 +63.06%  -11.00
290.00 351.75 292.34 -16.89%   61.75
119.40 112.70 292.29 +159.35%   -6.70
257.75 216.25 292.30 +35.17%  -41.50
371.50 336.50 353.20  +4.96%  -35.00
403.50 337.00 347.83  +3.21%  -66.50


# END

## 25 models
Create models for {5 contract months} x {5 backoffs}

In [408]:
def evaluate(model, inverse, X_test, y_test):
    y_predict = model.predict(X_test)
    p = []
    t = []
    for i in range(len(X_test)):
        p.append(inverse(y_predict[i]))
        t.append(inverse(y_test[i]))
       
    return dict(predict=p, test=t)

In [495]:
months = list(scaledtimeseries.keys())
window = 30
backoffs = [180, 120, 90, 60, 30]
model = Sequential()
model.add(LSTM(units=30, return_sequences=True, input_shape=(X_train.shape[1],2)))
model.add(LSTM(units=30, return_sequences=True))
model.add(LSTM(units=30))
model.add(Dense(units=2))
# model.summary()

for month in months:
    scaledtimeseries[month]['stats'] = {}
    for backoff in backoffs:
        X, y = mkTrainingData(scaledtimeseries, month, backoff, window)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.1)

        model.compile(optimizer='adam', loss='mean_squared_error')
        model.fit(X_train, y_train, epochs=100, batch_size=10)
        scaledtimeseries[month]['stats'][backoff] = evaluate(model, scaledtimeseries[month]['scaler'].inverse_transform, X_test, y_test)

Epoch 1/100
53/53 [==============================] - 14s 273ms/sample - loss: 0.5485
Epoch 2/100
53/53 [==============================] - 0s 4ms/sample - loss: 0.4326
Epoch 3/100
53/53 [==============================] - 0s 4ms/sample - loss: 0.3112
Epoch 4/100
53/53 [==============================] - 0s 4ms/sample - loss: 0.1709
Epoch 5/100
53/53 [==============================] - 0s 5ms/sample - loss: 0.0952
Epoch 6/100
53/53 [==============================] - 0s 6ms/sample - loss: 0.0881
Epoch 7/100
53/53 [==============================] - 0s 4ms/sample - loss: 0.0885
Epoch 8/100
53/53 [==============================] - 0s 4ms/sample - loss: 0.0724
Epoch 9/100
53/53 [==============================] - 0s 4ms/sample - loss: 0.0558
Epoch 10/100
53/53 [==============================] - 0s 4ms/sample - loss: 0.0443
Epoch 11/100
53/53 [==============================] - 0s 4ms/sample - loss: 0.0358
Epoch 12/100
53/53 [==============================] - 0s 4ms/sample - loss: 0.0282
Epoch 13/1

Epoch 97/100
53/53 [==============================] - 0s 4ms/sample - loss: 7.9580e-05
Epoch 98/100
53/53 [==============================] - 0s 4ms/sample - loss: 7.7044e-05
Epoch 99/100
53/53 [==============================] - 0s 4ms/sample - loss: 8.9596e-05
Epoch 100/100
53/53 [==============================] - 0s 4ms/sample - loss: 8.3961e-05
Epoch 1/100
54/54 [==============================] - 13s 245ms/sample - loss: 0.0100
Epoch 2/100
54/54 [==============================] - 0s 4ms/sample - loss: 0.0120
Epoch 3/100
54/54 [==============================] - 0s 4ms/sample - loss: 0.0092
Epoch 4/100
54/54 [==============================] - 0s 4ms/sample - loss: 0.0051
Epoch 5/100
54/54 [==============================] - 0s 4ms/sample - loss: 0.0028
Epoch 6/100
54/54 [==============================] - 0s 4ms/sample - loss: 0.0027
Epoch 7/100
54/54 [==============================] - 0s 4ms/sample - loss: 0.0023
Epoch 8/100
54/54 [==============================] - 0s 4ms/sample - loss:

54/54 [==============================] - 0s 4ms/sample - loss: 3.6831e-05
Epoch 87/100
54/54 [==============================] - 0s 4ms/sample - loss: 2.0829e-05
Epoch 88/100
54/54 [==============================] - 0s 4ms/sample - loss: 4.5964e-05
Epoch 89/100
54/54 [==============================] - 0s 4ms/sample - loss: 2.0209e-05
Epoch 90/100
54/54 [==============================] - 0s 4ms/sample - loss: 1.3059e-05
Epoch 91/100
54/54 [==============================] - 0s 5ms/sample - loss: 1.5047e-05
Epoch 92/100
54/54 [==============================] - 1s 17ms/sample - loss: 1.8967e-05
Epoch 93/100
54/54 [==============================] - 1s 12ms/sample - loss: 1.5210e-05
Epoch 94/100
54/54 [==============================] - 0s 8ms/sample - loss: 9.6921e-06
Epoch 95/100
54/54 [==============================] - 0s 4ms/sample - loss: 1.0577e-05
Epoch 96/100
54/54 [==============================] - 0s 4ms/sample - loss: 1.0925e-05
Epoch 97/100
54/54 [==============================] - 

54/54 [==============================] - 0s 4ms/sample - loss: 0.0060
Epoch 82/100
54/54 [==============================] - 0s 4ms/sample - loss: 0.0059
Epoch 83/100
54/54 [==============================] - 0s 4ms/sample - loss: 0.0056
Epoch 84/100
54/54 [==============================] - 0s 4ms/sample - loss: 0.0060
Epoch 85/100
54/54 [==============================] - 0s 4ms/sample - loss: 0.0037
Epoch 86/100
54/54 [==============================] - 0s 4ms/sample - loss: 0.0012
Epoch 87/100
54/54 [==============================] - 0s 4ms/sample - loss: 0.0013
Epoch 88/100
54/54 [==============================] - 0s 4ms/sample - loss: 9.2221e-04
Epoch 89/100
54/54 [==============================] - 0s 4ms/sample - loss: 2.9089e-04
Epoch 90/100
54/54 [==============================] - 0s 4ms/sample - loss: 1.5735e-04
Epoch 91/100
54/54 [==============================] - 0s 4ms/sample - loss: 2.6610e-04
Epoch 92/100
54/54 [==============================] - 0s 4ms/sample - loss: 1.1382e-

Epoch 76/100
54/54 [==============================] - 0s 4ms/sample - loss: 3.7776e-05
Epoch 77/100
54/54 [==============================] - 0s 5ms/sample - loss: 1.2888e-05
Epoch 78/100
54/54 [==============================] - 0s 9ms/sample - loss: 8.5249e-05
Epoch 79/100
54/54 [==============================] - 0s 4ms/sample - loss: 7.7460e-05
Epoch 80/100
54/54 [==============================] - 0s 4ms/sample - loss: 5.6348e-05
Epoch 81/100
54/54 [==============================] - 0s 4ms/sample - loss: 4.5322e-05
Epoch 82/100
54/54 [==============================] - 0s 4ms/sample - loss: 2.7028e-05
Epoch 83/100
54/54 [==============================] - 0s 4ms/sample - loss: 5.7990e-05
Epoch 84/100
54/54 [==============================] - 0s 4ms/sample - loss: 4.0022e-05
Epoch 85/100
54/54 [==============================] - 0s 4ms/sample - loss: 4.4691e-05
Epoch 86/100
54/54 [==============================] - 0s 4ms/sample - loss: 1.2510e-04
Epoch 87/100
54/54 [=======================

...

54/54 [==============================] - 0s 4ms/sample - loss: 9.3567e-05
Epoch 7/100
54/54 [==============================] - 0s 4ms/sample - loss: 1.2846e-04
Epoch 8/100
54/54 [==============================] - 0s 4ms/sample - loss: 1.2390e-04
Epoch 9/100
54/54 [==============================] - 0s 4ms/sample - loss: 4.7007e-05
Epoch 10/100
54/54 [==============================] - 0s 4ms/sample - loss: 3.6369e-05
Epoch 11/100
54/54 [==============================] - 0s 4ms/sample - loss: 2.1848e-05
Epoch 12/100
54/54 [==============================] - 0s 4ms/sample - loss: 2.4167e-05
Epoch 13/100
54/54 [==============================] - 0s 4ms/sample - loss: 2.9254e-05
Epoch 14/100
54/54 [==============================] - 0s 4ms/sample - loss: 1.3408e-05
Epoch 15/100
54/54 [==============================] - 0s 4ms/sample - loss: 1.5770e-05
Epoch 16/100
54/54 [==============================] - 0s 4ms/sample - loss: 1.4616e-05
Epoch 17/100
54/54 [==============================] - 0s 4m

54/54 [==============================] - 0s 4ms/sample - loss: 8.0290e-06
Epoch 95/100
54/54 [==============================] - 0s 4ms/sample - loss: 3.2992e-06
Epoch 96/100
54/54 [==============================] - 0s 4ms/sample - loss: 9.6607e-06
Epoch 97/100
54/54 [==============================] - 0s 4ms/sample - loss: 6.4940e-05
Epoch 98/100
54/54 [==============================] - 0s 4ms/sample - loss: 4.0607e-05
Epoch 99/100
54/54 [==============================] - 0s 4ms/sample - loss: 1.8180e-04
Epoch 100/100
54/54 [==============================] - 0s 4ms/sample - loss: 3.6108e-05
Epoch 1/100
54/54 [==============================] - 16s 304ms/sample - loss: 4.0323e-04
Epoch 2/100
54/54 [==============================] - 0s 4ms/sample - loss: 6.4929e-04
Epoch 3/100
54/54 [==============================] - 0s 4ms/sample - loss: 3.5894e-04
Epoch 4/100
54/54 [==============================] - 0s 4ms/sample - loss: 3.6191e-04
Epoch 5/100
54/54 [==============================] - 0s 

54/54 [==============================] - 0s 4ms/sample - loss: 2.4003e-04
Epoch 83/100
54/54 [==============================] - 0s 4ms/sample - loss: 1.6006e-04
Epoch 84/100
54/54 [==============================] - 0s 4ms/sample - loss: 1.8014e-04
Epoch 85/100
54/54 [==============================] - 0s 4ms/sample - loss: 1.4362e-04
Epoch 86/100
54/54 [==============================] - 0s 4ms/sample - loss: 8.6540e-05
Epoch 87/100
54/54 [==============================] - 0s 4ms/sample - loss: 6.1775e-05
Epoch 88/100
54/54 [==============================] - 0s 4ms/sample - loss: 2.0760e-05
Epoch 89/100
54/54 [==============================] - 0s 4ms/sample - loss: 3.4030e-05
Epoch 90/100
54/54 [==============================] - 0s 4ms/sample - loss: 2.8758e-05
Epoch 91/100
54/54 [==============================] - 0s 4ms/sample - loss: 1.8071e-05
Epoch 92/100
54/54 [==============================] - 0s 4ms/sample - loss: 9.3772e-06
Epoch 93/100
54/54 [==============================] - 0s

54/54 [==============================] - 0s 4ms/sample - loss: 7.8080e-06
Epoch 71/100
54/54 [==============================] - 0s 4ms/sample - loss: 8.4696e-06
Epoch 72/100
54/54 [==============================] - 0s 4ms/sample - loss: 1.6136e-05
Epoch 73/100
54/54 [==============================] - 0s 4ms/sample - loss: 2.0288e-05
Epoch 74/100
54/54 [==============================] - 0s 4ms/sample - loss: 2.7755e-05
Epoch 75/100
54/54 [==============================] - 0s 4ms/sample - loss: 3.9276e-05
Epoch 76/100
54/54 [==============================] - 0s 4ms/sample - loss: 3.2081e-05
Epoch 77/100
54/54 [==============================] - 0s 4ms/sample - loss: 3.6822e-05
Epoch 78/100
54/54 [==============================] - 0s 4ms/sample - loss: 5.6597e-05
Epoch 79/100
54/54 [==============================] - 0s 4ms/sample - loss: 3.8996e-05
Epoch 80/100
54/54 [==============================] - 0s 4ms/sample - loss: 6.6531e-05
Epoch 81/100
54/54 [==============================] - 0s

54/54 [==============================] - 0s 6ms/sample - loss: 1.7997e-04
Epoch 59/100
54/54 [==============================] - 0s 5ms/sample - loss: 1.6907e-04
Epoch 60/100
54/54 [==============================] - 0s 5ms/sample - loss: 1.2870e-04
Epoch 61/100
54/54 [==============================] - 0s 5ms/sample - loss: 1.3711e-04
Epoch 62/100
54/54 [==============================] - 0s 4ms/sample - loss: 1.0429e-04
Epoch 63/100
54/54 [==============================] - 0s 4ms/sample - loss: 5.8739e-05
Epoch 64/100
54/54 [==============================] - 0s 4ms/sample - loss: 1.2924e-04
Epoch 65/100
54/54 [==============================] - 0s 4ms/sample - loss: 9.4200e-05
Epoch 66/100
54/54 [==============================] - 0s 4ms/sample - loss: 3.8397e-05
Epoch 67/100
54/54 [==============================] - 0s 4ms/sample - loss: 1.7679e-05
Epoch 68/100
54/54 [==============================] - 0s 4ms/sample - loss: 1.9043e-05
Epoch 69/100
54/54 [==============================] - 0s

In [496]:
compare = lambda x, y: (x-y)/y*100

for month in scaledtimeseries:
    for backoff in scaledtimeseries[month]['stats']:
        print('#####\nMonth {:2d} backoff {:3d}'.format(month, backoff))
        p = scaledtimeseries[month]['stats'][backoff]['predict']
        t = scaledtimeseries[month]['stats'][backoff]['test']
        for i in range(len(p)):
            print('{:6.2f} {:6.2f} {:+5.2f}%'.format(p[i][0], t[i][0], compare(p[i][0], t[i][0])))

#scaledtimeseries[9]['stats'][180]

#####
Month 12 backoff 180
106.78 103.50 +3.17%
248.09 247.50 +0.24%
118.48 118.30 +0.15%
173.73 179.50 -3.22%
365.95 364.00 +0.54%
209.98 210.50 -0.25%
#####
Month 12 backoff 120
360.53 359.50 +0.29%
234.54 233.50 +0.45%
195.67 195.00 +0.34%
383.55 381.75 +0.47%
351.69 351.75 -0.02%
208.42 207.50 +0.44%
#####
Month 12 backoff  90
120.14 123.20 -2.49%
137.90 141.40 -2.47%
120.54 123.50 -2.40%
270.14 270.75 -0.23%
209.30 210.25 -0.45%
254.10 254.75 -0.25%
#####
Month 12 backoff  60
397.16 396.25 +0.23%
267.78 267.50 +0.10%
220.34 219.75 +0.27%
233.67 233.50 +0.07%
111.67 111.40 +0.25%
365.44 366.25 -0.22%
#####
Month 12 backoff  30
329.64 336.25 -1.97%
235.80 237.25 -0.61%
245.11 247.50 -0.97%
253.82 254.75 -0.36%
284.02 289.75 -1.98%
121.39 115.00 +5.56%
#####
Month  5 backoff 180
271.81 271.75 +0.02%
195.02 195.25 -0.12%
508.44 494.75 +2.77%
268.86 268.75 +0.04%
142.63 143.00 -0.26%
181.42 181.50 -0.04%
#####
Month  5 backoff 120
287.15 286.50 +0.23%
506.40 494.75 +2.35%
206.81 203.50

### scratch

In [351]:
model = Sequential()
model.add(LSTM(units=30, return_sequences=True, input_shape=(X_train.shape[1],2)))
model.add(LSTM(units=30, return_sequences=True))
model.add(LSTM(units=30))
model.add(Dense(units=2))
model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_26 (LSTM)               (None, 30, 30)            3960      
_________________________________________________________________
lstm_27 (LSTM)               (None, 30, 30)            7320      
_________________________________________________________________
lstm_28 (LSTM)               (None, 30)                7320      
_________________________________________________________________
dense_8 (Dense)              (None, 2)                 62        
Total params: 18,662
Trainable params: 18,662
Non-trainable params: 0
_________________________________________________________________


In [403]:
M = scaledtimeseries[9]
y2002 = M[2002]
y2002[(y2002['day']>=180) & (y2002['day']<210)]

,year,month,open,close,open_net,close_net,open_rel,close_rel,volume,openint,day,date
327,2002,9,225.75,225.25,-40.75,-41.25,0.847092,0.845216,1956,24552,209,2002-02-19
328,2002,9,225.75,225.25,-40.75,-41.25,0.847092,0.845216,1956,24552,208,2002-02-19
329,2002,9,225.75,225.25,-40.75,-41.25,0.847092,0.845216,1956,24552,207,2002-02-19
330,2002,9,225.75,225.25,-40.75,-41.25,0.847092,0.845216,1956,24552,206,2002-02-19
331,2002,9,226.00,226.50,-40.50,-40.00,0.848030,0.849906,635,24607,205,2002-02-20
332,2002,9,227.00,226.50,-39.50,-40.00,0.851782,0.849906,2041,25430,204,2002-02-21
333,2002,9,227.25,224.75,-39.25,-41.75,0.852720,0.843340,1986,26241,203,2002-02-22
334,2002,9,224.50,221.50,-42.00,-45.00,0.842402,0.831144,3339,29042,200,2002-02-25
335,2002,9,224.50,221.50,-42.00,-45.00,0.842402,0.831144,3339,29042,201,2002-02-25
336,2002,9,224.50,221.50,-42.00,-45.00,0.842402,0.831144,3339,29042,202,2002-02-25


In [229]:
X = []
y = []
month = 9
backoff = 180
window = 30
end_day = backoff + window

sept = timeseries[month]
years = sept.keys()


for year in years:

    t = []
    ts = sept[year]
    # grab close prices, then reverse order
    close_prices = ts[(ts['day']>=backoff) & (ts['day']<end_day)][['close']].iloc[::-1]['close']
    if len(close_prices) == window:
        X.extend(list(close_prices))
        y.append(ts.iloc[-1,3])

X, y = np.array(X), np.array(y)

In [371]:
months = pd.unique(raw['month'])

timeseries = {}
for month in months:
    timeseries[month] = {}
    years = pd.unique(raw[raw['month']==month]['year'])
    for year in years:
        ts = raw[(raw['year']==year) & (raw['month']==month)]
        if len(ts) > 0:
            ts = ts.copy()
            ts.sort_values(by=['date'], inplace=True)
            timeseries[month][year] = ts.reset_index(drop=True)

In [63]:
# remove identity columns
df = raw.drop(columns=['cik', 'week'])
# normalize ratio cols
df['Ar'] = df['Ar'].apply(lambda x: min(x, 1.0))
df['Dr'] = df['Dr'].apply(lambda x: min(x, 1.0))
#df.describe()

In [14]:
# create 3 output columns:
# 3at1 - 3% gain at 1 week
# 6at2 - 5% gain at 2 weeks
# 9at3 - 9% gain at 3 weeks
p0Idx = 12
df['3at1'] = (df.iloc[:,12+1]/df.iloc[:,12]) >= 1.03
df['6at2'] = (df.iloc[:,12+2]/df.iloc[:,12]) >= 1.06
df['9at3'] = (df.iloc[:,12+3]/df.iloc[:,12]) >= 1.09
df = df.drop(columns=['p0', 'p1', 'p2', 'p3', 'p4', 'p5'])

In [16]:
dflog = df.copy()
dflog['A'] = dflog['A'].apply(lambda x: np.log(x+1))
dflog['D'] = dflog['D'].apply(lambda x: np.log(x+1))
#dflog.describe()

### create model

In [45]:
X = dflog.iloc[:,:12]
y = dflog.iloc[:,12:]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [53]:
# define the keras model
model = Sequential()
model.add(Dense(24, input_dim=12, activation='relu'))
model.add(Dense(12, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.Precision()])

In [54]:
# fit the keras model on the dataset

model.fit(X, y['3at1'], epochs=20, batch_size=10)

Epoch 1/20
27296/27296 [==============================] - 4s 128us/sample - loss: 0.5246 - precision_1: 0.2468
Epoch 2/20
27296/27296 [==============================] - 4s 139us/sample - loss: 0.5148 - precision_1: 0.0000e+00
Epoch 3/20
27296/27296 [==============================] - 3s 127us/sample - loss: 0.5124 - precision_1: 0.0000e+00
Epoch 4/20
27296/27296 [==============================] - 3s 126us/sample - loss: 0.5119 - precision_1: 0.2000
Epoch 5/20
27296/27296 [==============================] - 4s 142us/sample - loss: 0.5114 - precision_1: 0.5000
Epoch 6/20
27296/27296 [==============================] - 3s 126us/sample - loss: 0.5112 - precision_1: 0.0000e+00
Epoch 7/20
27296/27296 [==============================] - 3s 127us/sample - loss: 0.5103 - precision_1: 0.0000e+00
Epoch 8/20
27296/27296 [==============================] - 3s 127us/sample - loss: 0.5104 - precision_1: 0.2500
Epoch 9/20
27296/27296 [==============================] - 3s 127us/sample - loss: 0.5101 - preci

In [129]:
y_test

array([[230.5 ],
       [211.75],
       [280.5 ],
       [354.75],
       [105.6 ]])

In [26]:
ts = scaledtimeseries[9][2002]
list(ts[ts['day']==30].iloc[0][['close', 'day']])

[273.0, 30]

In [448]:
M = scaledtimeseries[3]
for year in M:
    if isinstance(year, str):
        continue
    N = M[year]
    c = N[N['day']==0]['close'].to_list()[0]
    print('{:4d} {:6.2f}'.format(year, c))


1982 263.25
1976 268.75
1999 228.75
1980 262.00
1972 120.62
1967 139.30
1985 277.50
1987 161.00
1990 250.00
1991 242.50
1962 110.40
1977 254.75
1965 131.00
2000 228.75
1993 220.50
1964 119.30
1974 298.50
1998 259.25
1973 158.25
1992 267.50
1978 242.00
2001 211.50
1969 113.38
1961 113.00
1989 276.00
1966 123.60
2002 203.00
1971 146.25
1963 116.60
1970 124.38
1983 291.25
1960 113.70
1979 244.00
1995 241.25
1994 278.75
1986 234.25
1975 283.00
1968 119.40
1997 302.50
1981 351.50
1988 200.50
1984 350.00
1996 396.50


In [59]:
X_train[0]

array([[-0.14354858, -0.81363652],
       [-0.19129873, -0.80718573],
       [-0.19129873, -0.80073494],
       [-0.19129873, -0.79428415],
       [-0.21994882, -0.78783336],
       [-0.20275876, -0.78138257],
       [-0.21230879, -0.77493179],
       [-0.23522886, -0.768481  ],
       [-0.25241891, -0.76203021],
       [-0.25241891, -0.75557942],
       [-0.25241891, -0.74912863],
       [-0.22567883, -0.74267784],
       [-0.22567883, -0.73622705],
       [-0.22567883, -0.72977627],
       [-0.22949885, -0.72332548],
       [-0.22758884, -0.71687469],
       [-0.22758884, -0.7104239 ],
       [-0.22758884, -0.70397311],
       [-0.2485989 , -0.69752232],
       [-0.22758884, -0.69107153],
       [-0.23140885, -0.68462074],
       [-0.2142188 , -0.67816996],
       [-0.22567883, -0.67171917],
       [-0.22567883, -0.66526838],
       [-0.22567883, -0.65881759],
       [-0.25432892, -0.6523668 ],
       [-0.25432892, -0.64591601],
       [-0.24286889, -0.63946522],
       [-0.24477889,

In [47]:
l = X_train[:,:,0]
r = X_train[:,:,1]

In [58]:
l = l.reshape(l.shape[0], l.shape[1], 1)
r = r.reshape(r.shape[0], r.shape[1], 1)
j = np.concatenate((l, r), 2)
j[0]

array([[-0.14354858, -0.81363652],
       [-0.19129873, -0.80718573],
       [-0.19129873, -0.80073494],
       [-0.19129873, -0.79428415],
       [-0.21994882, -0.78783336],
       [-0.20275876, -0.78138257],
       [-0.21230879, -0.77493179],
       [-0.23522886, -0.768481  ],
       [-0.25241891, -0.76203021],
       [-0.25241891, -0.75557942],
       [-0.25241891, -0.74912863],
       [-0.22567883, -0.74267784],
       [-0.22567883, -0.73622705],
       [-0.22567883, -0.72977627],
       [-0.22949885, -0.72332548],
       [-0.22758884, -0.71687469],
       [-0.22758884, -0.7104239 ],
       [-0.22758884, -0.70397311],
       [-0.2485989 , -0.69752232],
       [-0.22758884, -0.69107153],
       [-0.23140885, -0.68462074],
       [-0.2142188 , -0.67816996],
       [-0.22567883, -0.67171917],
       [-0.22567883, -0.66526838],
       [-0.22567883, -0.65881759],
       [-0.25432892, -0.6523668 ],
       [-0.25432892, -0.64591601],
       [-0.24286889, -0.63946522],
       [-0.24477889,

In [141]:
class CustomScaler:
    '''scales the first `scaleCols` columns using scaler
    input is np.ndarray of shape [no_samples, width_of_a_sample, no_features]
    '''
    def __init__(self, scaler, scaleCols):
        self.scaler = scaler
        self.scaleCols = scaleCols

    def _apply(self, data, which):
        left = data[:, :, 0:self.scaleCols]
        right = data[:, :, self.scaleCols:]
        left2 = left.reshape(left.shape[0] * left.shape[1], left.shape[2])

        scaled = which(left)
        scaled = scaled.reshape(left.shape)
        return np.concatenate((scaled, right), 2)

    def transform(self, data):
        return self._apply(data, self.scaler.transform)

    def inverse_transform(self, data):
        return self._apply(data, self.scaler.inverse_transform)

In [142]:
cs = CustomScaler(scaledtimeseries[9]['scaler'], 2)

In [144]:
k = np.random.random_integers(0, 100, size=x.shape[0]*x.shape[1]*3)
k = k.reshape(x.shape[0], x.shape[1], 3)
print(k.shape)
j = np.concatenate((x, k), 2)
print(j.shape)
g = cs.inverse_transform(j)
print(g.shape)
print(g[0][0], j[0][0], x[0][0])

(54, 30, 3)
(54, 30, 5)
(54, 30, 5)
[273.5 120.   99.   94.   79. ] [-0.14354858 -0.81363652 99.         94.         79.        ] [-0.14354858 -0.81363652]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: This function is deprecated. Please call randint(0, 100 + 1) instead
  """Entry point for launching an IPython kernel.
